In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY="5g"
spark = SparkSession.builder.appName("taxi-fare-prediciton")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

21/12/20 04:41:46 WARN Utils: Your hostname, 6miniui-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.200.112 instead (on interface en0)
21/12/20 04:41:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/20 04:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
trips_df = spark.read.csv("/Users/6mini/trip/*", inferSchema=True, header=True)

In [4]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
trips_df.createOrReplaceTempView("trips")

In [6]:
# 팩터를 몇가지 더 추가한다.
query = """
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 5
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [7]:
data_df.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|              1|               142|                 43|          2.1|          0|     Friday|        11.8|
|              1|               238|                151|          0.2|          0|     Friday|         4.3|
|              1|               132|                165|         14.7|          0|     Friday|       51.95|
|              0|               138|                132|         10.6|          0|     Friday|       36.35|
|              1|                68|                 33|         4.94|          0|     Friday|       24.36|
|              1|               224|                 68|          1.6|          0|     Friday|       14.15|
|              1|           

In [8]:
data_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [9]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=6)

In [10]:
# 나뉘어진 데이터 프레임을 저장해둔다.
data_dir = "/Users/6mini/spark/taxi"

In [11]:
# 파퀫 형태로 저장
# 압축률이 좋고 디스크 io가 적다
# 컬럼 기반 포맷이다.
train_df.write.format("parquet").save(f"{data_dir}/train/")
test_df.write.format("parquet").save(f"{data_dir}/test/")

In [12]:
# 파퀫 형태로 불러오기
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

In [13]:
train_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [14]:
# 스트링 값을 숫자값으로 바꾸어 원핫인코딩을 진행한다.
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# 카테고리 피쳐 설정
cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

# 스테이지를 담는 배열
stages = []


for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c, outputCol= c + "_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c + "_onehot"])
    stages += [cat_indexer, onehot_encoder]

stages

[StringIndexer_ff1b1c4b32df,
 OneHotEncoder_4ea9b3272ed6,
 StringIndexer_65b49d3966df,
 OneHotEncoder_2e03a29c5802,
 StringIndexer_b6267b01a6ed,
 OneHotEncoder_5de1f0401482]

In [15]:
# 숫자형 전처리: 벡터 어셈블러, 스탠다드 스칼라
from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol= n + "_vecotr")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol= n + "_scaled")
    stages += [num_assembler, num_scaler]

stages

[StringIndexer_ff1b1c4b32df,
 OneHotEncoder_4ea9b3272ed6,
 StringIndexer_65b49d3966df,
 OneHotEncoder_2e03a29c5802,
 StringIndexer_b6267b01a6ed,
 OneHotEncoder_5de1f0401482,
 VectorAssembler_a56b59cc4ac2,
 StandardScaler_898a128ab6df,
 VectorAssembler_ab4e638f04bc,
 StandardScaler_73b16bd04318,
 VectorAssembler_7063855370da,
 StandardScaler_c4e3960fc997]

In [16]:
# 두가지를 프리프로세싱을 했는데 합치는 과정을 진행한다.
# 벡터 어셈블러를 이용하여 가능하다.
assembler_inputs = [c + "_onehot" for c in cat_feats] + [n + "_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="feature_vector")
stages += [assembler]

stages

[StringIndexer_ff1b1c4b32df,
 OneHotEncoder_4ea9b3272ed6,
 StringIndexer_65b49d3966df,
 OneHotEncoder_2e03a29c5802,
 StringIndexer_b6267b01a6ed,
 OneHotEncoder_5de1f0401482,
 VectorAssembler_a56b59cc4ac2,
 StandardScaler_898a128ab6df,
 VectorAssembler_ab4e638f04bc,
 StandardScaler_73b16bd04318,
 VectorAssembler_7063855370da,
 StandardScaler_c4e3960fc997,
 VectorAssembler_37481243c479]

In [17]:
# 스테이지로 파이프 라인 생성
from pyspark.ml import Pipeline

transform_stages = stages
pipeline = Pipeline(stages=transform_stages)
fitted_transformer = pipeline.fit(train_df)

In [18]:
vtrain_df = fitted_transformer.transform(train_df)

In [19]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    solver="normal",
    labelCol="total_amount",
    featuresCol="feature_vector"
)

In [20]:
vtrain_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vecotr: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vecotr: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vecotr: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nul

In [21]:
model = lr.fit(vtrain_df)

21/12/20 04:42:34 WARN Instrumentation: [f1853e0d] regParam is zero, which might cause numerical instability and overfitting.
21/12/20 04:42:37 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/12/20 04:42:37 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
21/12/20 04:42:38 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
21/12/20 04:42:38 WARN Instrumentation: [f1853e0d] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
21/12/20 04:42:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/12/20 04:42:39 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [22]:
vtest_df = fitted_transformer.transform(test_df)

In [23]:
predictions = model.transform(vtest_df)

In [24]:
# 캐싱을하여 나중에 쓰기 쉽게 만든다.
predictions.cache()

DataFrame[passenger_count: int, pickup_location_id: int, dropoff_location_id: int, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, passenger_count_vecotr: vector, passenger_count_scaled: vector, trip_distance_vecotr: vector, trip_distance_scaled: vector, pickup_time_vecotr: vector, pickup_time_scaled: vector, feature_vector: vector, prediction: double]

In [25]:
predictions.select(["trip_distance", "day_of_week", "total_amount", "prediction"]).show()

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|         0.01|     Sunday|       22.25|  12.3875521857505|
|          0.1|     Sunday|         3.3|10.744178027113051|
|          6.1|     Monday|        26.3|28.652704157860498|
|          7.1|     Monday|       35.76|31.174097556214498|
|          0.9|     Sunday|         8.3|  7.66950512061783|
|          2.7|     Sunday|        16.6|15.133976597996543|
|          1.0|   Saturday|         7.3| 9.111650395437803|
|          1.7|   Saturday|       14.75|12.508165587852046|
|          0.6|     Friday|         5.8| 9.667477937462271|
|          0.9|     Monday|        11.0|10.668972172418805|
|          1.1|     Sunday|         9.3|11.048364111576014|
|          0.2|     Sunday|         0.3| 7.720810167026536|
|          2.8|     Sunday|        12.3|15.966585384151795|
|          5.4|   Saturday|        24.3|

In [26]:
model.summary.rootMeanSquaredError

3.877648659368804

In [27]:
model.summary.r2

0.8922402466214665

In [28]:
spark.stop()